# Daily Saving Test

Test cách cắt nhiều ngày liên tục

In [3]:

import os
import cdsapi
from typing import Optional, Dict, Any
from dotenv import load_dotenv

class CDSAPIConfig:
    """Configuration class for CDS API with multiple credential sources"""
    
    def __init__(self, url: Optional[str] = None, key: Optional[str] = None, env_file: Optional[str] = None):
        """
        Initialize CDS API configuration
        
        Args:
            url: CDS API URL (optional, will use environment variable or default)
            key: CDS API key (optional, will use environment variable)
            env_file: Path to .env file to load (optional)
        """
        # Load .env file if specified
        if env_file:
            load_dotenv(env_file)
        else:
            # Try to load .env file from current directory
            load_dotenv()
        
        self.url = url or os.getenv('CDSAPI_URL', 'https://cds.climate.copernicus.eu/api')
        self.key = key or os.getenv('CDSAPI_KEY')
        
        if not self.key:
            raise ValueError("CDS API key is required. Set CDSAPI_KEY environment variable or pass key parameter.")
    
    def get_client(self) -> cdsapi.Client:
        """Create and return a configured CDS API client"""
        return cdsapi.Client(url=self.url, key=self.key)

In [ ]:
import xarray as xr
import rioxarray # Enables the .rio accessor
from pathlib import Path
import os

# --- 1. DEFINE YOUR PARAMETERS HERE ---

# Path to your downloaded NetCDF file
input_nc_file = Path('data/2024/01/era5_vietnam_2024_01_01.nc')

# Define the variable (band) you want to extract
# From your inspection, you can choose: 't2m', 'sp', 'skt', 'tp', etc.
variable_to_crop = 't2m' # Using 2m temperature as an example

# Define the specific date and time you want to select
timestamp_to_select = '2024-01-01T12:00:00' # Example: January 1st, 2024 at 12:00 UTC

# Define the bounding box [min_lon, min_lat, max_lon, max_lat]
# Example BBOX for the Hanoi area
hanoi_bbox = [105.5, 20.8, 106.2, 21.2] 

# Define the output path for your GeoTIFF file
output_tif_file = Path(f'output/hanoi_temperature_{variable_to_crop}_20240101_1200.tif')

# --- 2. THE PROCESSING SCRIPT ---

def crop_netcdf_to_geotiff(
    nc_file: Path,
    variable: str,
    timestamp: str,
    bbox: list,
    tif_file: Path
):
    """
    Loads a NetCDF file, crops a variable to a bounding box for a specific
    time, and saves it as a GeoTIFF.
    """
    if not nc_file.exists():
        print(f"Error: Input file not found at {nc_file}")
        return

    print(f"Processing {nc_file}...")
    
    try:
        # Ensure output directory exists
        tif_file.parent.mkdir(parents=True, exist_ok=True)
        
        # Open the dataset
        ds = xr.open_dataset(nc_file, engine='netcdf4')

        # --- Data Standardization ---
        # Rename 'valid_time' to 'time' for consistency
        if 'valid_time' in ds.coords:
            ds = ds.rename({'valid_time': 'time'})

        # Handle 'expver' dimension if it exists
        if 'expver' in ds.dims:
            ds = ds.sel(expver=1)
            
        # Select the specific data array (band)
        if variable not in ds.data_vars:
            print(f"Error: Variable '{variable}' not found in the dataset.")
            print(f"Available variables: {list(ds.data_vars.keys())}")
            return
            
        data_array = ds[variable]
        
        # --- Time and Space Selection ---
        # Select the data for the specific timestamp
        print(f"Selecting data for time: {timestamp}")
        data_at_time = data_array.sel(time=timestamp, method='nearest')
        
        # --- Geospatial Operations with rioxarray ---
        # 1. Set the CRS (Coordinate Reference System) for the data
        # ERA5 data uses WGS84, which corresponds to EPSG:4326
        if data_at_time.rio.crs is None:
            data_at_time = data_at_time.rio.set_crs("EPSG:4326")
            print("Set Coordinate Reference System (CRS) to EPSG:4326 (WGS84)")

        # 2. Clip the data to the bounding box
        min_lon, min_lat, max_lon, max_lat = bbox
        print(f"Clipping to bounding box: {bbox}")
        clipped_data = data_at_time.rio.clip_box(
            minx=min_lon, miny=min_lat, maxx=max_lon, maxy=max_lat
        )
        
        # --- Save to GeoTIFF ---
        # The .rio.to_raster function handles the georeferencing
        print(f"Saving cropped data to {tif_file}...")
        clipped_data.rio.to_raster(tif_file, tiled=True, compress='LZW')
        
        print("Successfully created GeoTIFF:")
        print(f"   - Path: {tif_file}")
        print(f"   - Shape: {clipped_data.shape}")
        print(f"   - CRS: {clipped_data.rio.crs}")

        ds.close()

    except Exception as e:
        print(f"An error occurred: {e}")


# --- 3. RUN THE FUNCTION ---
if __name__ == "__main__":
    crop_netcdf_to_geotiff(
        nc_file=input_nc_file,
        variable=variable_to_crop,
        timestamp=timestamp_to_select,
        bbox=hanoi_bbox,
        tif_file=output_tif_file
    )

Processing data/2024/01/era5_vietnam_2024_01_01.nc...
Selecting data for time: 2024-01-01T12:00:00
Set Coordinate Reference System (CRS) to EPSG:4326 (WGS84)
Clipping to bounding box: [105.5, 20.8, 106.2, 21.2]
Saving cropped data to output/hanoi_temperature_t2m_20240101_1200.tif...
Successfully created GeoTIFF:
   - Path: output/hanoi_temperature_t2m_20240101_1200.tif
   - Shape: (5, 8)
   - CRS: EPSG:4326


/tmp/ipykernel_70584/749318814.py:77: FutureWarning: It is recommended to use 'rio.write_crs()' instead. 'rio.set_crs()' will likelybe removed in a future release.
  data_at_time = data_at_time.rio.set_crs("EPSG:4326")


In [5]:
import xarray as xr
from pathlib import Path

# --- Inspection Code ---

# Choose one of the files that your download script successfully created
# For example, the one for January 1st, 2024
file_to_inspect = Path('data/2024/01/era5_vietnam_2024_01_01.nc')

if file_to_inspect.exists():
    print(f"\n--- Inspecting File: {file_to_inspect} ---")
    try:
        # Open the dataset
        ds_inspect = xr.open_dataset(file_to_inspect, engine='netcdf4')

        print("\nDataset Dimensions:")
        print(ds_inspect.dims)

        print("\nDataset Coordinates:")
        print(ds_inspect.coords)

        print("\nDataset Data Variables (Bands):")
        for var_name in ds_inspect.data_vars:
            print(f"- {var_name} (Dimensions: {ds_inspect[var_name].dims})")
            
        print("\nFull Dataset Info:")
        print(ds_inspect)

        ds_inspect.close() # Close the dataset to free up resources
        print("\n--- Inspection Complete ---")

    except Exception as e:
        print(f"Error opening or inspecting {file_to_inspect}: {e}")
else:
    print(f"Error: File not found at {file_to_inspect}. Please ensure it was downloaded successfully.")

# --- End of Inspection Code ---


--- Inspecting File: data/2024/01/era5_vietnam_2024_01_01.nc ---

Dataset Dimensions:
FrozenMappingWarningOnValuesAccess({'valid_time': 24, 'latitude': 156, 'longitude': 81})

Dataset Coordinates:
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 192B 2024-01-01 ... 2024-01-01T23...
  * latitude    (latitude) float64 1kB 23.5 23.4 23.3 23.2 ... 8.3 8.2 8.1 8.0
  * longitude   (longitude) float64 648B 102.0 102.1 102.2 ... 109.8 109.9 110.0
    expver      (valid_time) <U4 384B ...

Dataset Data Variables (Bands):
- d2m (Dimensions: ('valid_time', 'latitude', 'longitude'))
- t2m (Dimensions: ('valid_time', 'latitude', 'longitude'))
- skt (Dimensions: ('valid_time', 'latitude', 'longitude'))
- slhf (Dimensions: ('valid_time', 'latitude', 'longitude'))
- ssr (Dimensions: ('valid_time', 'latitude', 'longitude'))
- sshf (Dimensions: ('valid_time', 'latitude', 'longitude'))
- ssrd (Dimensions: ('valid_time', 'latitude', 'longitude'))
- pev (Dimensions: ('

In [ ]:
import zipfile
import xarray as xr
import numpy as np
from era5_data_download import CDSAPIConfig
import shutil
import os
import calendar # To get the number of days in each month
from pathlib import Path # Import Path for easier directory management

# Initialize CDS API client
c = CDSAPIConfig(env_file='.env').get_client()

year = 2024 # You can change this to any year you need

# Define the base output directory
base_output_dir = Path('data')

print(f"Starting ERA5-Land data download for year {year} (daily files)...")
print("-" * 60)

for month in range(1, 13): # Loop through all 12 months
    # Determine the number of days in the current month
    _, num_days = calendar.monthrange(year, month)

    # Define the output directory for the current month and create it if it doesn't exist
    month_output_dir = base_output_dir / str(year) / f"{month:02d}"
    month_output_dir.mkdir(parents=True, exist_ok=True)
    print(f"Ensured directory: {month_output_dir}")

    for day in range(1, num_days + 1): # Loop through all days in the month
        # Define output filenames with the new directory structure
        output_filename_raw = month_output_dir / f'era5_vietnam_raw_{year}_{month:02d}_{day:02d}.nc'
        output_filename_extracted = month_output_dir / f'era5_vietnam_{year}_{month:02d}_{day:02d}.nc'

        print(f"Downloading data for {year}-{month:02d}-{day:02d} to {month_output_dir}...")

        try:
            c.retrieve(
                'reanalysis-era5-land',
                #########################################################
                # Sai ten bands
                {
                    'format': 'netcdf',
                    'product_type': 'reanalysis',
                    'variable': [
                        "2m_dewpoint_temperature",
                        "2m_temperature",
                        "skin_temperature",
                        "surface_latent_heat_flux",
                        "surface_net_solar_radiation",
                        "surface_sensible_heat_flux",
                        "surface_solar_radiation_downwards",
                        "potential_evaporation",
                        "total_evaporation",
                        "10m_u_component_of_wind",
                        "10m_v_component_of_wind",
                        "surface_pressure",
                        "total_precipitation"
                        ],
                    'year': str(year),
                    'month': f'{month:02d}',
                    'day': [f'{day:02d}'], # Request single day
                    'time': [f'{h:02d}:00' for h in range(0, 24)],
                    'area': [23.5, 102.0, 8.0, 110.0],  # Vietnam bounding box
                    # 'grid': [0.1, 0.1] # 0.1 degree resolution (~11km) - uncomment if needed
                },
                str(output_filename_raw) # Convert Path object to string for retrieve
            )
            print(f"Download complete: {output_filename_raw}")

            # Auto-extract ZIP if needed
            if zipfile.is_zipfile(output_filename_raw):
                print(f"Extracting ZIP file: {output_filename_raw}...")
                with zipfile.ZipFile(output_filename_raw, 'r') as zip_ref:
                    nc_files = [f for f in zip_ref.namelist() if f.endswith('.nc')]
                    if nc_files:
                        # Extract the first .nc file found in the zip to the current directory
                        zip_ref.extract(nc_files[0], path=month_output_dir) # Extract to month_output_dir
                        
                        # Move the extracted file to its final unique filename within the month directory
                        extracted_temp_path = month_output_dir / nc_files[0]
                        shutil.move(extracted_temp_path, output_filename_extracted)
                        print(f"Extracted to {output_filename_extracted}")
                    else:
                        print(f"No NetCDF file found inside {output_filename_raw}")
                # Remove the raw zip file after extraction
                os.remove(output_filename_raw)
            else:
                # If it's already a .nc file, and not a zip, just rename it
                shutil.move(output_filename_raw, output_filename_extracted)
                print(f"Downloaded directly as NetCDF: {output_filename_extracted}")

        except Exception as e:
            print(f"Error processing {year}-{month:02d}-{day:02d}: {e}")
            continue # Continue to the next day even if one fails

print("-" * 60)
print(f"Finished processing all data for year {year}.")

In [6]:
import xarray as xr
import rioxarray # Enables the .rio accessor
from pathlib import Path
import numpy as np

# --- 1. DEFINE YOUR PARAMETERS ---

# Path to one of your downloaded NetCDF files
# Make sure this file exists from your previous downloads
file_to_inspect = Path('data/2024/01/era5_vietnam_2024_01_01.nc')

# --- 2. CODE TO CHECK RESOLUTION ---

if file_to_inspect.exists():
    print(f"\n--- Checking Resolution for: {file_to_inspect} ---")
    try:
        # Open the dataset
        ds = xr.open_dataset(file_to_inspect, engine='netcdf4')

        # Rename 'valid_time' to 'time' if present for consistency
        if 'valid_time' in ds.coords:
            ds = ds.rename({'valid_time': 'time'})

        # If 'expver' dimension exists, select the first experiment version
        if 'expver' in ds.dims:
            ds = ds.sel(expver=1)
            
        print("\n--- Resolution in Degrees ---")
        if 'latitude' in ds.coords and 'longitude' in ds.coords:
            lat_res_deg = abs(ds.latitude.values[1] - ds.latitude.values[0])
            lon_res_deg = abs(ds.longitude.values[1] - ds.longitude.values[0])

            print(f"Latitude (N-S) Resolution: {lat_res_deg:.4f} degrees")
            print(f"Longitude (E-W) Resolution: {lon_res_deg:.4f} degrees")
            print(f"This matches the 0.1 degree grid you observed in metadata.")
        else:
            print("Latitude or Longitude coordinates not found in dataset.")

        print("\n--- Approximate Resolution in Meters ---")
        # Approximate conversion: 1 degree latitude is approx 111.32 km (111320 meters)
        # Longitude resolution varies with latitude.
        
        # Get a central latitude for better approximation for Vietnam
        if 'latitude' in ds.coords:
            central_lat = ds.latitude.mean().values
            
            # Constants for WGS84 ellipsoid (more accurate)
            a = 6378137.0  # Semi-major axis (equatorial radius)
            b = 6356752.314245179 # Semi-minor axis (polar radius)
            
            # M_d = a (1-e^2) / (1-e^2 sin^2(phi))^3/2   (meridional radius of curvature, N-S)
            # N_p = a / (1-e^2 sin^2(phi))^1/2           (prime vertical radius of curvature, E-W)
            # e^2 = (a^2 - b^2) / a^2
            
            e_squared = (a**2 - b**2) / a**2
            
            lat_rad = np.radians(central_lat)
            
            # Calculate meters per degree for latitude (N-S)
            meridional_radius_of_curvature = a * (1 - e_squared) / (1 - e_squared * np.sin(lat_rad)**2)**1.5
            meters_per_degree_lat = np.pi * meridional_radius_of_curvature / 180
            
            # Calculate meters per degree for longitude (E-W)
            parallel_radius_of_curvature = a * np.cos(lat_rad) / np.sqrt(1 - e_squared * np.sin(lat_rad)**2)
            meters_per_degree_lon = np.pi * parallel_radius_of_curvature / 180

            approx_lat_res_m = lat_res_deg * meters_per_degree_lat
            approx_lon_res_m = lon_res_deg * meters_per_degree_lon

            print(f"At central latitude {central_lat:.2f}°N (approx. for Vietnam):")
            print(f"  Approx. Latitude (N-S) Resolution: {approx_lat_res_m:.2f} meters (~{approx_lat_res_m/1000:.2f} km)")
            print(f"  Approx. Longitude (E-W) Resolution: {approx_lon_res_m:.2f} meters (~{approx_lon_res_m/1000:.2f} km)")
            print("\nNote: The longitude resolution is approximate and varies slightly across your area.")
            print("For practical purposes, 0.1 degree ERA5-Land is often considered ~11 km.")

            # --- Using rioxarray for geospatial information ---
            # Set CRS if not already set by rioxarray (it often auto-detects for ERA5)
            if ds.rio.crs is None:
                ds.rio.set_crs("EPSG:4326", inplace=True)
            
            print(f"\n--- Using rioxarray (.rio accessor) ---")
            print(f"rioxarray identified CRS: {ds.rio.crs}")
            print(f"rioxarray calculated resolution (y, x): {ds.rio.resolution()}")
            
        ds.close()

    except Exception as e:
        print(f"Error opening or inspecting {file_to_inspect}: {e}")
else:
    print(f"Error: File not found at {file_to_inspect}. Please ensure it was downloaded successfully.")

print("\nAll bands (e.g., t2m, tp, u10, v10) in this file share the same resolution.")


--- Checking Resolution for: data/2024/01/era5_vietnam_2024_01_01.nc ---

--- Resolution in Degrees ---
Latitude (N-S) Resolution: 0.1000 degrees
Longitude (E-W) Resolution: 0.1000 degrees
This matches the 0.1 degree grid you observed in metadata.

--- Approximate Resolution in Meters ---
At central latitude 15.75°N (approx. for Vietnam):
  Approx. Latitude (N-S) Resolution: 11065.61 meters (~11.07 km)
  Approx. Longitude (E-W) Resolution: 10716.65 meters (~10.72 km)

Note: The longitude resolution is approximate and varies slightly across your area.
For practical purposes, 0.1 degree ERA5-Land is often considered ~11 km.

--- Using rioxarray (.rio accessor) ---
rioxarray identified CRS: EPSG:4326
rioxarray calculated resolution (y, x): (0.1, -0.1)

All bands (e.g., t2m, tp, u10, v10) in this file share the same resolution.


/tmp/ipykernel_60121/1691989572.py:79: FutureWarning: It is recommended to use 'rio.write_crs()' instead. 'rio.set_crs()' will likelybe removed in a future release.
  ds.rio.set_crs("EPSG:4326", inplace=True)


In [ ]:
import xarray as xr
from pathlib import Path
import numpy as np

# --- Parameters (matching your previous script) ---
file_to_inspect = Path('data/2024/01/era5_vietnam_2024_01_01.nc')
timestamp_to_select = '2024-01-01T12:00:00'

# --- Verification Code ---

if file_to_inspect.exists():
    print(f"\n--- Detailed Time Selection Check for: {file_to_inspect} ---")
    try:
        ds = xr.open_dataset(file_to_inspect, engine='netcdf4')

        # Rename 'valid_time' to 'time' for consistency
        if 'valid_time' in ds.coords:
            ds = ds.rename({'valid_time': 'time'})

        print("\nAll time values in the dataset (first 5 and last 5):")
        print(ds.time.values[:5])
        print("...")
        print(ds.time.values[-5:])

        print(f"\nAttempting to select time: {timestamp_to_select}")
        selected_time_value = ds.time.sel(time=timestamp_to_select, method='nearest').item()
        print(f"Time value actually selected by 'nearest': {selected_time_value}")

        # Find the index of the selected time value
        try:
            # Convert to Python datetime for easier comparison
            python_datetimes = ds.time.dt.round('h').to_numpy().tolist() # Round to hour to avoid ns precision issues
            target_dt = np.datetime64(timestamp_to_select).astype('datetime64[h]').item() # Round target as well

            # Find exact index
            actual_index = python_datetimes.index(target_dt)
            print(f"The exact index for {selected_time_value} is: {actual_index}")
        except ValueError:
            # If the exact time (after rounding) isn't in the list
            print(f"The exact time {selected_time_value} was not found in the dataset's time coordinate list (after rounding to hour).")
            # Fallback to finding the index of the nearest
            actual_index = np.argmin(np.abs(ds.time.values - np.datetime64(timestamp_to_select)))
            print(f"Closest index by absolute difference: {actual_index}")

        # Verify the time at that index
        print(f"Time value at index {actual_index}: {ds.time.values[actual_index]}")

        ds.close()

    except Exception as e:
        print(f"An error occurred during time selection check: {e}")
else:
    print(f"Error: File not found at {file_to_inspect}.")



--- Detailed Time Selection Check for: data/2024/01/era5_vietnam_2024_01_01.nc ---

All time values in the dataset (first 5 and last 5):
['2024-01-01T00:00:00.000000000' '2024-01-01T01:00:00.000000000'
 '2024-01-01T02:00:00.000000000' '2024-01-01T03:00:00.000000000'
 '2024-01-01T04:00:00.000000000']
...
['2024-01-01T19:00:00.000000000' '2024-01-01T20:00:00.000000000'
 '2024-01-01T21:00:00.000000000' '2024-01-01T22:00:00.000000000'
 '2024-01-01T23:00:00.000000000']

Attempting to select time: 2024-01-01T12:00:00
Time value actually selected by 'nearest': 1704110400000000000
The exact time 1704110400000000000 was not found in the dataset's time coordinate list (after rounding to hour).
Closest index by absolute difference: 12
Time value at index 12: 2024-01-01T12:00:00.000000000


/media/ekai2/data2tb/datmh/Platform/era5_land_test/venv/lib/python3.10/site-packages/xarray/core/accessor_dt.py:170: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  field_values = method(freq=freq).values


# Monthly saving test

Test xem tải đc ko
Test xem có cắt ngày(ví dụ 2-3 ngày liên tục đc ko)